In [2]:
import zipfile
import sqlite3

import pandas as pd
import numpy as np
import sys
import importlib.util

import os

os.chdir('rubiks-cube-NxNxN-solver')
# Print the current working directory
print("Current Working Directory: ", os.getcwd())

database_file = '../../solutions.db'
solution_method = "dwalton76 NxNxN algo"

Current Working Directory:  /Users/seanbearden/PycharmProjects/Kaggle/src/santa-2023/magic-cube/rubiks-cube-NxNxN-solver


In [3]:
with zipfile.ZipFile('../../../../res/data/santa-2023.zip', 'r') as z:
    
    with z.open('puzzle_info.csv') as f:
        puzzle_info = pd.read_csv(f, index_col = 'puzzle_type')        
                
    with z.open('puzzles.csv') as f:
        puzzles = pd.read_csv(f, index_col='id')
    
    with z.open('sample_submission.csv') as f:
        submission = pd.read_csv(f)

In [3]:
U = ['U', 'F', 'R', 'B', 'L', 'D']
U_dict = {
    'A': 'U',
    'B': 'F',
    'C': 'R',
    'D': 'B',
    'E': 'L',
    'F': 'D',
}

def state2ubl(state):
    state_split = state.split(';')
    dim = int(np.sqrt(len(state_split) // 6))
    dim_2 = dim**2
    
    s = ''.join([U_dict[f] for f in state_split])
    
    return s[:dim_2] + s[2*dim_2:3*dim_2] + s[dim_2:2*dim_2] + s[5*dim_2:] + s[4*dim_2:5*dim_2] + s[3*dim_2:4*dim_2]


In [56]:
def relabel_NxNxN(state, checkered=False):        
    # Split the string into individual elements.
    state_list = state.split(';')
    dim = int(np.sqrt(len(state_list) // 6))
    
    dim_2 = dim**2
    faces = []
    for i in range((dim-2)**2):
        faces += list(range((i//(dim-2) + 1) * dim + i % 2 + 1, 6*dim_2, dim_2))
    
    edges = []
    for i in range(dim-2): #each cube on edge
        edges.append((0 * dim_2 + i + 1,                        3 * dim_2 + (dim - 1) - (i + 1)))
        edges.append((0 * dim_2 + (dim - 1) * dim +  i + 1,     1 * dim_2 + (i + 1)))        
        edges.append((0 * dim_2 + dim * (i + 1),                4 * dim_2 + (i + 1)))        
        edges.append((0 * dim_2 + dim * (i + 1) + dim - 1,      2 * dim_2 + (dim - 1) - (i + 1)))
        
        edges.append((5 * dim_2 + i + 1,                        1 * dim_2 + (dim - 1) * dim + (i + 1)))
        edges.append((5 * dim_2 + (dim - 1) * dim +  i + 1,     4 * dim_2 - i - 2))        
        edges.append((5 * dim_2 + dim * (i + 1),                5 * dim_2 - i - 2))        
        edges.append((5 * dim_2 + dim * (i + 1) + dim - 1,      2 * dim_2 + (dim - 1) * dim + (i + 1)))
        
        edges.append((3 * dim_2 + dim * (i + 1) + dim - 1,      4 * dim_2 + dim * (i + 1)))
        edges.append((2 * dim_2 + dim * (i + 1) + dim - 1,      3 * dim_2 + dim * (i + 1)))        
        edges.append((1 * dim_2 + dim * (i + 1),                4 * dim_2 + dim * (i + 1) + dim - 1))   # backwards? 
        
        edges.append((0 * dim_2 + dim * (i + 1) + dim - 1,      2 * dim_2 + dim * (i + 1)))
        
                
    corners = [
        (dim_2 - 1, dim_2 + dim - 1, 2 * dim_2), 
        (dim - 1, 2 * dim_2 + dim -1, 3 * dim_2),
        (0, 3 * dim_2 + dim - 1, 4 * dim_2),
        ((dim - 1) * dim, dim_2 , 4 * dim_2 + dim - 1),         
        (dim_2 + (dim - 1) * dim, 5 * dim_2 - 1, 5 * dim_2),        
        (2*dim_2 - 1, 3 * dim_2 - dim, 5 * dim_2 + dim -1), 
        (3*dim_2 - 1, 4*dim_2 - dim, 6*dim_2 - 1), 
        (4*dim_2-1, 5*dim_2-dim, 6*dim_2 - dim)
    ]
    

    
    if state[0] == 'N':
        for i, val in enumerate(state_list):
            # Extract the number part from the string like 'N33' -> 33.
            num = int(val[1:])
            # print(i, num)
            # Replace based on the given criteria.
            if 0 <= num < dim_2:
                state_list[i] = 'A'
            elif dim_2 <= num < 2*dim_2:
                state_list[i] = 'B'
            elif 2*dim_2 <= num < 3*dim_2:
                state_list[i] = 'C'
            elif 3*dim_2 <= num < 4*dim_2:
                state_list[i] = 'D'
            elif 4*dim_2 <= num < 5*dim_2:
                state_list[i] = 'E'
            elif 5*dim_2 <= num < 6*dim_2:
                state_list[i] = 'F'
    elif checkered:
        face2color = {
            0: 'A',
            1: 'B',
            2: 'C',
            3: 'D',
            4: 'E',
            5: 'F',
        }
        print(faces)
        for idx in faces:
            # a face labeled B could be B or A
            state_list[idx] = face2color[idx // dim_2]
            
    return ';'.join(state_list)  

state = 'A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;B;C;B;C;B;C;B;C;B;C;B;C;B;C;B;C;C;D;C;D;C;D;C;D;C;D;C;D;C;D;C;D;D;E;D;E;D;E;D;E;D;E;D;E;D;E;D;E;E;F;E;F;E;F;E;F;E;F;E;F;E;F;E;F;F;A;F;A;F;A;F;A;F;A;F;A;F;A;F;A'
relabel_NxNxN(state, checkered=True)


[5, 21, 37, 53, 69, 85, 6, 22, 38, 54, 70, 86, 9, 25, 41, 57, 73, 89, 10, 26, 42, 58, 74, 90]


'A;B;A;B;A;A;A;B;A;A;A;B;A;B;A;B;B;C;B;C;B;B;B;C;B;B;B;C;B;C;B;C;C;D;C;D;C;C;C;D;C;C;C;D;C;D;C;D;D;E;D;E;D;D;D;E;D;D;D;E;D;E;D;E;E;F;E;F;E;E;E;F;E;E;E;F;E;F;E;F;F;A;F;A;F;F;F;A;F;F;F;A;F;A;F;A'

In [55]:
dim = 4
dim_2 = dim**2
i = 3
list(range(dim * (dim - 1 - i//(dim-2) - 1) + i + 1, 6*dim_2, dim_2))

0 * dim_2 + dim * (i//(dim-2)) + dim + i + 1

(i//(dim-2) + 1) * dim + i % 2 + 1

10

In [5]:
# import subprocess
# 
# # Define the list of commands or arguments you want to run in Bash.
# commands = [
#     './rubiks-cube-NxNxN-solver/rubiks-cube-solver.py --state '
#     'FUUURFRDURUBDBLURDLRUULRFLLFRUBLBRUFDUFRUFDRRBUDDLRBDFLFLBDDLLBFDDBLLDBRDBRLUDFFBBFLFBUBLDRFURFB',
#     # Add more commands as needed.
# ]
# 
# # Loop through the commands, run each in Bash, and collect the output.
# for command in commands:
#     process = subprocess.run(command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
#     # stdout will have the standard output of the command, and stderr will have the standard error.
#     output = process.stdout
#     error = process.stderr
#     
#     # Do something with the output and error.
#     print("Output:", output)
#     if error:
#         print("Error:", error)


In [63]:
# Connect to the SQLite database
conn = sqlite3.connect(database_file)
cursor = conn.cursor()


outputs = {}
for id, row in puzzles.iterrows():
    if row['puzzle_type'][:4] == 'cube':
        
        dim = int(row['puzzle_type'].split('/')[-1])
        if id not in range(200, 201):
            continue
        print(id) 
        moves = eval(puzzle_info.loc[row['puzzle_type'], 'allowed_moves'])
        for move in list(moves):
            moves['-'+move] = np.argsort(moves[move]).tolist()
        
        M = {}
        M["U"] = f'-d{dim-1}'
        M["R"] = "r0"
        M["B"] = f"-f{dim-1}"
        M["F"] = "f0"
        M["L"] = f"-r{dim-1}"
        M["D"] = "d0"
        
        if dim > 3:
            M["Uw"] = f'-d{dim-2}.-d{dim-1}'
            M["Rw"] = f"r0.r1"
            M["Bw"] = f'-f{dim-2}.-f{dim-1}'
            M["Fw"] = f"f0.f1"
            M["Lw"] = f'-r{dim-2}.-r{dim-1}'
            M["Dw"] = f"d0.d1"
        if dim >= 6:
            M["2Uw"] = f'-d{dim-2}.-d{dim-1}'
            M["2Rw"] = f"r0.r1"
            M["2Bw"] = f'-f{dim-2}.-f{dim-1}'
            M["2Fw"] = f"f0.f1"
            M["2Lw"] = f'-r{dim-2}.-r{dim-1}'
            M["2Dw"] = f"d0.d1"
            
            width_max = dim // 2
            for i in range(3, width_max + 1):
                M[f"{i}Uw"] = f'-d{dim-i}.' + M[f"{i-1}Uw"]
                M[f"{i}Rw"] = M[f"{i-1}Rw"] + f'.r{i-1}'
                M[f"{i}Bw"] = f'-f{dim-i}.' + M[f"{i-1}Bw"]
                M[f"{i}Fw"] = M[f"{i-1}Fw"] + f'.f{i-1}'
                M[f"{i}Lw"] = f'-r{dim-i}.' + M[f"{i-1}Lw"]
                M[f"{i}Dw"] = M[f"{i-1}Dw"] + f'.d{i-1}'
    
            
        # print(M)
        for m in list(M):
            M[m+"2"] = M[m] + '.' + M[m]
            if "-" in M[m]:
                M[m+"'"] = M[m].replace("-","")
            else:
                M[m+"'"] = '.'.join(["-"+i for i in M[m].split('.')])
        # id += 1
        # print(dim)
        init_state = row['initial_state']
        sol_state = row['solution_state']
        init_state = relabel_NxNxN(init_state)
        sol_state = relabel_NxNxN(sol_state)
        init_state = sol_state
        state = state2ubl(init_state)

        
        if sol_state[:2*dim**2-1] == ';'.join(['A']*dim**2) or True:
            print(f'Starting {id}')
            output = !./rubiks-cube-solver.py --state $state
            outputs[id] = output
            if output[-1][:9] == 'Solution:':
                print(output[-1])
                sol = output[-1].split(': ')[1]
            else:
                for n in range(1, 21):
                    if 'Solution:' in output[-n]:
                        sol = output[-n].split('Solution: ')[1].split('2023-')[0]
                        break
                
            # print(sol)
            mmoves = '.'.join([M[m] for m in sol.split(' ')])
            
            new_state = init_state
            for move in mmoves.split('.'):
                new_state = ';'.join(list(np.asarray(new_state.split(';'))[np.array(moves[move])]))
            print(new_state)
            
            # I = ['r0.r1.r2','d0.d1.d2','f0.f1.f2']
            I = ['.'.join([f'{j}{i}' for i in range(dim)]) for j in ['r', 'd', 'f']]
            
            for init_moves in [''] + I + [i1 + '.' + i2 for i1 in I for i2 in I]+ [i1 + '.' + i2+ '.' + i3 for i1 in I for i2 in I for i3 in I]+ [i1 + '.' + i2+ '.' + i3 + '.' + i4 for i1 in I for i2 in I for i3 in I for i4 in I]:
                temp_state = new_state
                if len(init_moves) > 0:
                    for move in init_moves.split('.'):
                        temp_state = ';'.join(list(np.asarray(temp_state.split(';'))[np.array(moves[move])]))
                if temp_state == sol_state:
                    print(f'solved id: {id}')
                    if len(init_moves) > 0:
                        mmoves += '.' + init_moves
                    break
                    
            # validation
            state = row['initial_state'].split(";")
            for move_name in mmoves.split('.'):
                state = [state[i] for i in moves[move_name]]
            # print(puzzle["solution_state"])
            # print(state)
            # print(mmoves)
            try:
                assert row['solution_state'].split(";") == state
                
                mmoves_length = len(mmoves.split('.'))
                
                select_query = "SELECT count FROM solutions WHERE id = ?"
    
                # Execute the query
                cursor.execute(select_query, (id,))
                response = cursor.fetchone()
                best_moves_length = response[0]
                
                if mmoves_length < best_moves_length:
                    # Insert the moves into the database
                    insert_query = ("INSERT OR REPLACE INTO solutions (id, moves, count, solution_method) VALUES (?, ?, ?, ?)")
                    cursor.execute(insert_query, (id, mmoves, mmoves_length, solution_method))
                    conn.commit()
            except AssertionError:
                print(f"assertion error for {id}")
                print(row['solution_state'])
                print(';'.join(state))

                break

# Commit the changes and close the connection
conn.commit()
conn.close()


200
Starting 200
B;F;A;A;C;F;F;A;D;D;C;D;F;E;D;C;B;E;D;C;A;E;A;C;E;C;B;F;D;F;C;B;A;B;E;E;C;C;A;B;B;E;B;B;F;B;C;B;E;D;A;D;F;D;F;C;F;F;A;C;C;D;D;F;D;A;B;F;C;C;E;E;B;B;D;F;E;E;E;A;D;F;A;A;E;B;E;A;F;A;D;B;E;D;A;C
assertion error for 200
A;B;A;B;A;B;A;B;A;B;A;B;A;B;A;B;B;C;B;C;B;C;B;C;B;C;B;C;B;C;B;C;C;D;C;D;C;D;C;D;C;D;C;D;C;D;C;D;D;E;D;E;D;E;D;E;D;E;D;E;D;E;D;E;E;F;E;F;E;F;E;F;E;F;E;F;E;F;E;F;F;A;F;A;F;A;F;A;F;A;F;A;F;A;F;A
['B', 'E', 'A', 'C', 'D', 'F', 'F', 'D', 'C', 'D', 'B', 'C', 'D', 'A', 'B', 'B', 'D', 'D', 'F', 'D', 'A', 'A', 'C', 'B', 'D', 'A', 'B', 'C', 'A', 'B', 'F', 'F', 'D', 'A', 'B', 'A', 'D', 'D', 'B', 'A', 'B', 'F', 'D', 'F', 'E', 'C', 'A', 'A', 'C', 'E', 'E', 'F', 'E', 'B', 'A', 'A', 'B', 'C', 'C', 'E', 'B', 'F', 'F', 'C', 'F', 'D', 'C', 'A', 'B', 'A', 'D', 'C', 'E', 'E', 'E', 'D', 'C', 'A', 'F', 'E', 'E', 'C', 'F', 'E', 'E', 'E', 'C', 'C', 'F', 'F', 'E', 'D', 'B', 'B', 'E', 'F']


In [64]:
output
# init_state

['2023-12-27 08:51:45,046     rubiks-cube-solver.py:26     INFO: rubiks-cube-solver.py begin',
 '2023-12-27 08:51:45,064               __init__.py:1468     INFO: ',
 '',
 'Initial Cube',
 '============',
 '         \x1bU\x1b \x1bF\x1b \x1bU\x1b \x1bF\x1b ',
 '         \x1bU\x1b \x1bF\x1b \x1bU\x1b \x1bF\x1b ',
 '         \x1bU\x1b \x1bF\x1b \x1bU\x1b \x1bF\x1b ',
 '         \x1bU\x1b \x1bF\x1b \x1bU\x1b \x1bF\x1b ',
 '',
 '\x1bL\x1b \x1bD\x1b \x1bL\x1b \x1bD\x1b  \x1bF\x1b \x1bR\x1b \x1bF\x1b \x1bR\x1b  \x1bR\x1b \x1bB\x1b \x1bR\x1b \x1bB\x1b  \x1bB\x1b \x1bL\x1b \x1bB\x1b \x1bL\x1b ',
 '\x1bL\x1b \x1bD\x1b \x1bL\x1b \x1bD\x1b  \x1bF\x1b \x1bR\x1b \x1bF\x1b \x1bR\x1b  \x1bR\x1b \x1bB\x1b \x1bR\x1b \x1bB\x1b  \x1bB\x1b \x1bL\x1b \x1bB\x1b \x1bL\x1b ',
 '\x1bL\x1b \x1bD\x1b \x1bL\x1b \x1bD\x1b  \x1bF\x1b \x1bR\x1b \x1bF\x1b \x1bR\x1b  \x1bR\x1b \x1bB\x1b \x1bR\x1b \x1bB\x1b  \x1bB\x1b \x1bL\x1b \x1bB\x1b \x1bL\x1b ',
 '\x1bL\x1b \x1bD\x1b \x1bL\x1b \x1bD\x1b  \x1bF\x1b \x1bR\x1b \x1bF\x1

In [11]:
 total_incorrect = 0
for i, j in zip(row['solution_state'].split(";"), state):
    print(i, j, i==j)
    if i!=j:
        total_incorrect += 1
print(total_incorrect)

A N0 False
A N1 False
A N2 False
A N3 False
A N4 False
D N5 False
D N8 False
D N13 False
D N6 False
D N9 False
G N10 False
G N17 False
G N12 False
G N11 False
G N14 False
J N15 False
J N16 False
J N7 False
J N18 False
J N19 False
M N20 False
M N21 False
M N22 False
M N23 False
M N24 False
P N25 False
P N26 False
P N27 False
P N28 False
P N29 False
S N30 False
S N41 False
S N38 False
S N31 False
S N34 False
V N35 False
V N42 False
V N37 False
V N36 False
V N39 False
Y N40 False
Y N43 False
Y N32 False
Y N33 False
Y N44 False
b N45 False
b N46 False
b N47 False
b N48 False
b N49 False
A N50 False
A N51 False
A N52 False
A N53 False
A N54 False
D N55 False
D N68 False
D N63 False
D N58 False
D N59 False
G N60 False
G N61 False
G N62 False
G N57 False
G N64 False
J N65 False
J N66 False
J N67 False
J N56 False
J N69 False
M N70 False
M N71 False
M N72 False
M N73 False
M N74 False
P N75 False
P N76 False
P N77 False
P N78 False
P N79 False
S N80 False
S N81 False
S N82 False
S N91 False
S 

In [9]:
# import importlib
# import brute_force
# importlib.reload(brute_force)
# 
# puzzle = puzzles.loc[id]
# sol = brute_force.solve(puzzle, puzzle_info, ';'.join(state))
# sol

In [10]:
sol

"Rw2 Uw B Bw2 Rw' L Dw L2 Lw2 Fw' Fw2 Dw B' R Bw2 Dw B2 Uw' Rw2 U L Fw2 R' U' U2 F Uw2 Lw2 D2 Bw2 Fw2 R2 F2 Dw2 B2 R Uw2 L Bw2 F' Rw2 D2 B Rw2 B' R2 F' Uw2 B2 L2 Lw2 Uw2 B Lw2 D2 B D' R' B L' D2 R' U' B2 D F2 R2 L2 B' L2 F2 R2 D2 L2"

In [4]:
submission.loc[126]

id                                                     126
moves    f1.d1.f1.d2.f0.f2.-r2.-r2.d2.f0.f2.-r2.r1.-f2....
Name: 126, dtype: object